In [1]:
import numpy as np, pandas as pd, string
from string import digits
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
lines = pd.read_csv("/content/Hindi_English_Truncated_Corpus.csv", encoding='utf-8')
lines = lines[lines['source'] == 'ted'][['english_sentence', 'hindi_sentence']].dropna().drop_duplicates()
lines = lines.sample(n=25000, random_state=42)

In [3]:
def clean_text(text):
    exclude = set(string.punctuation)
    text = ''.join(ch for ch in text if ch not in exclude)
    text = text.translate(str.maketrans('', '', digits))
    return text.strip().lower()

In [4]:
lines['english_sentence'] = lines['english_sentence'].apply(clean_text)
lines['hindi_sentence'] = lines['hindi_sentence'].apply(clean_text)
lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x: 'start_ ' + x + ' _end')

In [5]:
eng_tokenizer = Tokenizer()
eng_tokenizer.fit_on_texts(lines['english_sentence'])
eng_seq = eng_tokenizer.texts_to_sequences(lines['english_sentence'])

hin_tokenizer = Tokenizer(filters='')
hin_tokenizer.fit_on_texts(lines['hindi_sentence'])
hin_seq = hin_tokenizer.texts_to_sequences(lines['hindi_sentence'])

In [6]:
max_eng_len = max(len(seq) for seq in eng_seq)
max_hin_len = max(len(seq) for seq in hin_seq)

encoder_input = pad_sequences(eng_seq, maxlen=max_eng_len, padding='post')
decoder_input = pad_sequences(hin_seq, maxlen=max_hin_len, padding='post')

In [7]:
decoder_target = np.zeros((decoder_input.shape[0], decoder_input.shape[1], 1))
decoder_target[:, 0:-1, 0] = decoder_input[:, 1:]

### Encoder

In [11]:
encoder_inputs = Input(shape=(None,))
eng_vocab_size = len(eng_tokenizer.word_index) + 1
enc_emb = Embedding(eng_vocab_size, 256)(encoder_inputs)
enc_outputs, state_h, state_c = LSTM(256, return_state=True)(enc_emb)
encoder_states = [state_h, state_c]

In [13]:
decoder_inputs = Input(shape=(None,))
hin_vocab_size = len(hin_tokenizer.word_index) + 1
dec_emb_layer = Embedding(hin_vocab_size, 256)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(hin_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [14]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
model.fit([encoder_input, decoder_input], decoder_target, batch_size=64, epochs=20, validation_split=0.2)

Epoch 1/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 24s 60ms/step - loss: 3.2264 - val_loss: 2.0264
Epoch 2/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 18s 56ms/step - loss: 2.0148 - val_loss: 1.9916
Epoch 3/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 21s 57ms/step - loss: 1.9650 - val_loss: 1.9608
Epoch 4/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 22s 61ms/step - loss: 1.9262 - val_loss: 1.9248
Epoch 5/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 19s 57ms/step - loss: 1.8961 - val_loss: 1.9085
Epoch 6/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 57ms/step - loss: 1.8591 - val_loss: 1.8761
Epoch 7/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 19s 61ms/step - loss: 1.8335 - val_loss: 1.8607
Epoch 8/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 21s 61ms/step - loss: 1.8190 - val_loss: 1.8392
Epoch 9/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 18s 57ms/step - loss: 1.7834 - val_loss: 1.8310
Epoch 10/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 18s 57ms/step - loss: 1.7717 - val_loss: 1.8163
Epoch 11/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 21s 61ms/step - loss: 1.7607 - val_loss: 1.8006
Epoch 12/20
313/313 ━━━━━━━━━━

In [15]:
encoder_model_inf = Model(encoder_inputs, encoder_states)

In [17]:
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
dec_inf_emb = dec_emb_layer(decoder_inputs)
dec_outputs_inf, state_h_inf, state_c_inf = decoder_lstm(dec_inf_emb, initial_state=decoder_states_inputs)
decoder_outputs_inf = decoder_dense(dec_outputs_inf)
decoder_model_inf = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs_inf, state_h_inf, state_c_inf])

In [18]:
reverse_eng = {v: k for k, v in eng_tokenizer.word_index.items()}
reverse_hin = {v: k for k, v in hin_tokenizer.word_index.items()}

In [32]:
def translate(sentence):
    sentence = clean_text(sentence)
    seq = eng_tokenizer.texts_to_sequences([sentence])
    padded = pad_sequences(seq, maxlen=max_eng_len, padding='post')
    states = encoder_model_inf.predict(padded)

    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = hin_tokenizer.word_index['start_']

    decoded = []
    while True:
        output, h, c = decoder_model_inf.predict([target_seq] + states)
        token_index = np.argmax(output[0, -1, :])
        word = reverse_hin.get(token_index, '')

        if word == '_end' or len(decoded) >= max_hin_len:
            break

        decoded.append(word)
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = token_index
        states = [h, c]

    return ' '.join(decoded)

print("English: one new year ")
print("Hindi:", translate("one new year"))

English: one new year 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Hindi: एक अलग साल में
